In [1]:
%matplotlib inline
import sys, os
sys.path.append('../..')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

In [22]:
from sktime.base import load
#from sklearn.model_selection import GridSearchCV
#from sktime.classification.feature_based import TSFreshClassifier
from sktime.classification.deep_learning.cnn import CNNClassifier

In [23]:
period = 20
filepath = os.path.join('..','..','data','trend index','training set', f'trend_{period}day.hdf')
file = h5py.File(filepath, mode='r')

In [24]:
ohlc  = file['ohlc'][:]
trend = file['trend'][:]
# shuffling
idx = np.random.permutation(len(ohlc))
ohlc = ohlc[idx]
trend = trend[idx]

print(ohlc.shape)
print(trend.shape)


(1139, 4, 20)
(1139,)


In [25]:
file.close()

### 1. Train data & Test data 생성 (7:3)

In [26]:
idx = int(len(ohlc)*0.7)
ohlc_train = ohlc[:idx]
trend_train = trend[:idx]

ohlc_test = ohlc[idx:]
trend_test = trend[idx:]

print(ohlc_train.shape)
print(ohlc_test.shape)

(797, 4, 20)
(342, 4, 20)


### 2. 훈련 모델 생성 및 훈련

In [30]:
network = CNNClassifier(n_epochs=100,batch_size=4, verbose=True)
#network = TSFreshClassifier(verbose=True)
network.fit(ohlc_train, trend_train)

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 20, 4)]           0         
                                                                 
 conv1d_10 (Conv1D)          (None, 20, 6)             174       
                                                                 
 average_pooling1d_10 (Avera  (None, 6, 6)             0         
 gePooling1D)                                                    
                                                                 
 conv1d_11 (Conv1D)          (None, 6, 12)             516       
                                                                 
 average_pooling1d_11 (Avera  (None, 2, 12)            0         
 gePooling1D)                                                    
                                                                 
 flatten_5 (Flatten)         (None, 24)                0   

C:\Users\yosep\python\miniconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


200/200 [==============================] - 1s 822us/step - loss: 0.1684 - accuracy: 0.5910
Epoch 2/100
200/200 [==============================] - 0s 837us/step - loss: 0.1339 - accuracy: 0.7365
Epoch 3/100
200/200 [==============================] - 0s 817us/step - loss: 0.1101 - accuracy: 0.7967
Epoch 4/100
200/200 [==============================] - 0s 837us/step - loss: 0.0992 - accuracy: 0.8068
Epoch 5/100
200/200 [==============================] - 0s 837us/step - loss: 0.0866 - accuracy: 0.8243
Epoch 6/100
200/200 [==============================] - 0s 832us/step - loss: 0.0804 - accuracy: 0.8419
Epoch 7/100
200/200 [==============================] - 0s 837us/step - loss: 0.0760 - accuracy: 0.8444
Epoch 8/100
200/200 [==============================] - 0s 842us/step - loss: 0.0690 - accuracy: 0.8620
Epoch 9/100
200/200 [==============================] - 0s 837us/step - loss: 0.0670 - accuracy: 0.8708
Epoch 10/100
200/200 [==============================] - 0s 842us/step - loss: 0.0661 

Epoch 80/100
200/200 [==============================] - 0s 832us/step - loss: 0.0215 - accuracy: 0.9699
Epoch 81/100
200/200 [==============================] - 0s 832us/step - loss: 0.0206 - accuracy: 0.9711
Epoch 82/100
200/200 [==============================] - 0s 832us/step - loss: 0.0233 - accuracy: 0.9598
Epoch 83/100
200/200 [==============================] - 0s 837us/step - loss: 0.0210 - accuracy: 0.9649
Epoch 84/100
200/200 [==============================] - 0s 832us/step - loss: 0.0209 - accuracy: 0.9611
Epoch 85/100
200/200 [==============================] - 0s 832us/step - loss: 0.0216 - accuracy: 0.9636
Epoch 86/100
200/200 [==============================] - 0s 832us/step - loss: 0.0228 - accuracy: 0.9649
Epoch 87/100
200/200 [==============================] - 0s 832us/step - loss: 0.0196 - accuracy: 0.9674
Epoch 88/100
200/200 [==============================] - 0s 842us/step - loss: 0.0182 - accuracy: 0.9749
Epoch 89/100
200/200 [==============================] - 0s 837us

CNNClassifier(batch_size=4, n_epochs=100, verbose=True)

In [31]:
network.score(ohlc_test, trend_test)

86/86 [==============================] - 0s 587us/step


0.8538011695906432

In [32]:
network.predict_proba(ohlc_test)

86/86 [==============================] - 0s 597us/step


array([[1.4869191e-01, 8.5112846e-01, 1.7967998e-04],
       [1.9814562e-02, 9.8002923e-01, 1.5622462e-04],
       [9.7562754e-01, 2.4264438e-02, 1.0794546e-04],
       ...,
       [7.1589113e-08, 3.1556992e-04, 9.9968439e-01],
       [3.5212142e-05, 9.8508006e-01, 1.4884687e-02],
       [9.9963862e-01, 2.7490128e-04, 8.6455628e-05]], dtype=float32)

### 4. 검증

In [33]:
from tools.instruments import instruments
from tools.datatools import norm
from tools.display import highchart
quotes = instruments.quotes(fields='ohlc')
symbols = list(quotes.columns.levels[0])

In [34]:
%load_ext autoreload
%aimport tools.datatools
%aimport tools.display
%autoreload 2

In [37]:
symbol = np.random.choice(symbols)
quote = quotes['W'].dropna()
idx = 500#np.random.randint(0,len(quote)-51)
ohlc = quote.iloc[idx:idx+period]
data = norm(ohlc.values)
data = np.moveaxis(np.array([data]),1,2 )
print(network.predict_proba(data).round(2))
highchart([{'data':ohlc, 'type':'candlestick'}])

1/1 [==============================] - 0s 15ms/step
[[0. 1. 0.]]


### 3.모델을 파일로 저장

In [63]:
savepath = os.path.join('..','..','data','trend index','models', f'trend_{period}day')
network.save(savepath)

INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_50day\keras\assets


INFO:tensorflow:Assets written to: ..\..\data\trend index\models\trend_50day\keras\assets


<zipfile.ZipFile filename='..\\..\\data\\trend index\\models\\trend_50day.zip' mode='r'>